In [1]:
import os
import copy
import pybars
import json
from manejo_base_de_datos import baseDatos
from manejo_archivos_json import dataJson
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
manejadorBaseDeDatos=baseDatos("postgres","carlos98","127.0.0.1","5432","chatblocks")#manejador de consultas a base de datos 
manejadorArchivos = dataJson(os.getcwd()+"/") 

In [2]:
def compiladorPybars(source,jsonData):
    compiler = pybars.Compiler()
    template = compiler.compile(source)
    output = template(jsonData)
    json_acceptable_string = output.replace("'", "\"")
    datos = json.loads(json_acceptable_string)
    return datos

In [3]:
def cargarVariable(datosVariable,id_robot):
    datosVariable.update({"id_robot":id_robot})
    variable = manejadorBaseDeDatos.insertDatos(datosVariable["variable"],"variables")
    return int(variable["id_var"])
    

In [4]:
def obtenerDatosQR(datosJson):    
    contenido=datosJson["content"]
    opciones=list()
    outputs=list()
    for i in range(len(datosJson["buttons"])):
        opciones.append(datosJson["buttons"][i]["title"])
        outputs.append(datosJson["buttons"][i]["next_block"])
    datos={"contenido":contenido,
           "opciones":",".join(opciones),
           "next_id":",".join(outputs)        
         }
    return datos

In [5]:
def obtenerDatosSlide(bloque):    
    datosBloqueSlide = {"namestate":bloque["name"],"blocktype":bloque["type"],"typingtime":bloque["parameters"]["typingtime"],"default_id":bloque["outputs"]["default_id"],"id_var":bloque["parameters"]["id_var"],"id_robot":bloque["parameters"]["id_robot"]}
    bloqueSlide  = manejadorBaseDeDatos.insertDatos(datosBloqueSlide,"bloqueslide")
    idBloqueSlide = bloqueSlide["id_block"]    
    elementos = copy.deepcopy(bloque["parameters"]["json"])
    next_id=list()
    for elemento in range(len(elementos)):
        datosElemento = copy.deepcopy(elementos[elemento])
        datosElemento.update({"blocktype":bloque["type"]})
        datosElemento.update({"id_block":idBloqueSlide})
        del datosElemento["buttons"]   
        datosElementoBase =  manejadorBaseDeDatos.insertDatos(datosElemento,"elementos")
        idElemento=datosElementoBase["id_elements"]                
        botones=copy.deepcopy(elementos[elemento]["buttons"])
        for b in range(len(botones)):
            datosBoton = copy.deepcopy(botones[b])
            datosBoton.update({"id_elemento":str(idElemento)})
            datosBoton.update({"contentbutton":botones[b]["opc_nextid"]})                    
            datosBoton={"id_elemento":str(idElemento),"titlebutton":botones[b]["titlebutton"],"typebutton":botones[b]["typebutton"],"contentbutton":botones[b]["opc_nextid"],"opc_nextid":botones[b]["opc_nextid"]}
            boton = manejadorBaseDeDatos.insertDatos(datosBoton,"botones") 
            next_id.append(botones[b]["opc_nextid"])
    next_id=",".join(next_id)
    tabla = "bloqueslide"
    datosActualizar={"next_id":next_id}
    datos={"id_block":idBloqueSlide}            
    tabla = "bloqueslide"   
    manejadorBaseDeDatos.update(datosActualizar,datos,tabla)   

In [6]:
botData=manejadorArchivos.abrirJson("chatbotFB.json") #archivo con datos de la aplicación de facebook
respuesta = manejadorBaseDeDatos.getDatos(botData,"robots")
hayRobot = True
if "error" not in respuesta:
    datosRobot = {"id_robot", respuesta["id_robot"]}
else:
    hayRobot = False

función select


In [2]:
manejadorBaseDeDatos.borrarBaseDatos()

In [8]:
if not hayRobot:
    datosBot= manejadorBaseDeDatos.insertDatos(botData,"robots")
    idBot=int(datosBot["id_robot"])
else:
    idBot=respuesta["id_robot"]
    datosBot = {"id_robot": idBot}
bloquesBot = manejadorArchivos.abrirJson("bloquesBotCorreo.json") # archivo con bloques del sistema
for bloque,datosBloque in bloquesBot.items():
    datosBloque["parameters"].update({"id_robot":idBot})   
manejadorArchivos.jsonUnico("bloquesBotCorreo.json", bloquesBot)

In [9]:
if hayRobot:
    manejadorBaseDeDatos.borrarBaseDatosPorRobot(str(datosBot["id_robot"]))
    print(datosBot["id_robot"])

380


In [10]:
if hayRobot:
    print(bloquesBot.keys())
    for bot in bloquesBot.keys():    
        bloque = dict() 
        if bloquesBot[bot]["parameters"].get("jsonVariable"):  
            print(bloquesBot[bot]["parameters"]["jsonVariable"]["variable"])
            bloquesBot[bot]["parameters"]["jsonVariable"]["variable"].update({"id_robot":bloquesBot[bot]["parameters"]["id_robot"]})    
            print(type(bloquesBot[bot]["parameters"]["jsonVariable"]["variable"]))
            variable = manejadorBaseDeDatos.insertDatos(bloquesBot[bot]["parameters"]["jsonVariable"]["variable"],"variables")         
            bloquesBot[bot]["parameters"].update({"id_var":int(variable["id_var"])})  
            del bloquesBot[bot]["parameters"]["jsonVariable"]
        if bloquesBot[bot]["parameters"].get("nameVariable"):
            datos=dict()
            datos.update({"id_robot":bloquesBot[bot]["parameters"].get("id_robot")})
            datos.update({"name_var":bloquesBot[bot]["parameters"].get("nameVariable")})
            print(datos)
            print(type(datos))
            variable = manejadorBaseDeDatos.getDatos(datos,"variables")
            bloquesBot[bot]["parameters"].update({"id_var":int(variable["id_var"])})
            del bloquesBot[bot]["parameters"]["nameVariable"] 

        if bloquesBot[bot]["type"] == "quickReply": 
            del bloquesBot[bot]["parameters"]["content"]
            del bloquesBot[bot]["parameters"]["buttonTrue"]
            del bloquesBot[bot]["parameters"]["buttonFalse"]
            del bloquesBot[bot]["parameters"]["typeVisual"]
            del bloquesBot[bot]["parameters"]["handlebars"]       
            QR = obtenerDatosQR(bloquesBot[bot]["parameters"]["json"])  
            bloque.update(QR)
            del bloquesBot[bot]["parameters"]["json"]       

        elif bloquesBot[bot]["type"] == "slide":
            del bloquesBot[bot]["parameters"]["content"]
            del bloquesBot[bot]["parameters"]["buttonTrue"]
            del bloquesBot[bot]["parameters"]["buttonFalse"]
            del bloquesBot[bot]["parameters"]["typeVisual"]
            del bloquesBot[bot]["parameters"]["handlebars"]
            slide= obtenerDatosSlide(bloquesBot[bot])
            del bloquesBot[bot]["parameters"]["json"]  
        elif bloquesBot[bot]["type"] == "nlp":
            dataAux = copy.deepcopy( bloquesBot[bot]["parameters"]["json_nlp"])
            bloquesBot[bot]["parameters"]["json_nlp"] = str(json.dumps(dataAux))
            print("\n")
            print( bloquesBot[bot]["parameters"]["json_nlp"] )
        for salida in bloquesBot[bot]["outputs"]:
            if type(bloquesBot[bot]["outputs"][salida]) != list:
                bloque.update({salida:bloquesBot[bot]["outputs"][salida]})
        bloque.update({"namestate":bloquesBot[bot]["name"]})
        bloque.update(bloquesBot[bot]["parameters"])
        bloque.update({"blocktype":bloquesBot[bot]["type"]})    
        if bloquesBot[bot]["type"] == "slide":
            pass
        else:
            print(bloquesBot[bot]["type"].lower())
            manejadorBaseDeDatos.insertDatos(bloque,"bloque"+bloquesBot[bot]["type"].lower())
else:      
    print("else")
    for bot in bloquesBot.keys():    
        bloque = dict() 
        if bloquesBot[bot]["parameters"].get("jsonVariable"):  
            print(bloquesBot[bot]["parameters"]["jsonVariable"]["variable"])
            bloquesBot[bot]["parameters"]["jsonVariable"]["variable"].update({"id_robot":bloquesBot[bot]["parameters"]["id_robot"]})    
            print(type(bloquesBot[bot]["parameters"]["jsonVariable"]["variable"]))
            variable = manejadorBaseDeDatos.insertDatos(bloquesBot[bot]["parameters"]["jsonVariable"]["variable"],"variables")         
            bloquesBot[bot]["parameters"].update({"id_var":int(variable["id_var"])})  
            del bloquesBot[bot]["parameters"]["jsonVariable"]
        if bloquesBot[bot]["parameters"].get("nameVariable"):
            datos=dict()
            datos.update({"id_robot":bloquesBot[bot]["parameters"].get("id_robot")})
            datos.update({"name_var":bloquesBot[bot]["parameters"].get("nameVariable")})
            print(datos)
            print(type(datos))
            variable = manejadorBaseDeDatos.getDatos(datos,"variables")
            bloquesBot[bot]["parameters"].update({"id_var":int(variable["id_var"])})
            del bloquesBot[bot]["parameters"]["nameVariable"] 

        if bloquesBot[bot]["type"] == "quickReply": 
            del bloquesBot[bot]["parameters"]["content"]
            del bloquesBot[bot]["parameters"]["buttonTrue"]
            del bloquesBot[bot]["parameters"]["buttonFalse"]
            del bloquesBot[bot]["parameters"]["typeVisual"]
            del bloquesBot[bot]["parameters"]["handlebars"]       
            QR = obtenerDatosQR(bloquesBot[bot]["parameters"]["json"])  
            bloque.update(QR)
            del bloquesBot[bot]["parameters"]["json"]       

        elif bloquesBot[bot]["type"] == "slide":
            del bloquesBot[bot]["parameters"]["content"]
            del bloquesBot[bot]["parameters"]["buttonTrue"]
            del bloquesBot[bot]["parameters"]["buttonFalse"]
            del bloquesBot[bot]["parameters"]["typeVisual"]
            del bloquesBot[bot]["parameters"]["handlebars"]
            slide= obtenerDatosSlide(bloquesBot[bot])
            del bloquesBot[bot]["parameters"]["json"]  
        elif bloquesBot[bot]["type"] == "nlp":
            dataAux = copy.deepcopy( bloquesBot[bot]["parameters"]["json_nlp"])
            bloquesBot[bot]["parameters"]["json_nlp"] = str(json.dumps(dataAux))
            print("\n")
            print( bloquesBot[bot]["parameters"]["json_nlp"] )
        for salida in bloquesBot[bot]["outputs"]:
            if type(bloquesBot[bot]["outputs"][salida]) != list:
                bloque.update({salida:bloquesBot[bot]["outputs"][salida]})
        bloque.update({"namestate":bloquesBot[bot]["name"]})
        bloque.update(bloquesBot[bot]["parameters"])
        bloque.update({"blocktype":bloquesBot[bot]["type"]})    
        if bloquesBot[bot]["type"] == "slide":
            pass
        else:        
            manejadorBaseDeDatos.insertDatos(bloque,"bloque"+bloquesBot[bot]["type"].lower())
        
print("bot cargado con exito ")
        
    

dict_keys(['prueba_inicio', 'prueba_qa', 'prueba_dos', 'pruebaCorreo_01inicial_inicial'])
informativo
informativo
informativo
{'name_var': 'var_case', 'opc_type': 'Variable', 'var': 0, 'opc_data': 'Numerico'}
<class 'dict'>
quickreply
bot cargado con exito 
